<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Matías López
- Nombre de alumno 2: José Saffie


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/JoseSaffie/MDS7202-1-Primavera-2023`

# Importamos librerias útiles

In [4]:
#!pip install -qq xgboost optuna

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\josei\AppData\Local\Temp\ipykernel_21312\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [6]:
# Inserte su código acá

# Primero hacemos una copia del df

df2 = df.copy()

# 1.
# Separamos los datos en los tres conjuntos
train_data, test_data = train_test_split(df2, test_size=0.3, random_state=42)
validation_data, test_data = train_test_split(test_data, test_size=0.33, random_state=42)

# 2.
# Creamos la funcion para extraer el mes y su informacion
def extract_date(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

# 3. 
# Implementar ColumnTransformer y la transformacion de fecha
date_transformer = FunctionTransformer(extract_date)
numeric_features = ['lat','long','pop','price']
categorical_features = ['city','shop','brand','container','capacity']

# Creamos el pipe line de transformación
preprocessor = ColumnTransformer(
    transformers=[
        ('date', date_transformer, ['date']),
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

#4.
# Pipeline para modelo de dummies
pipeline_dummy = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

# 5.
# Entrenamos el pipeline
train_x = train_data.drop('quantity', axis=1)
pipeline_dummy.fit(train_x, train_data['quantity'])
predictions_dummy = pipeline_dummy.predict(validation_data)

# Calculamos el error absoluto y lo guardamos
mae_dummy = mean_absolute_error(validation_data['quantity'], predictions_dummy)


# 6.
# Lo mismo pero con xgb
pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

train_x = train_data.drop('quantity', axis=1)
pipeline_xgb.fit(train_x, train_data['quantity'])
predictions_xgb = pipeline_xgb.predict(validation_data)

# lo guardamos
mae_xgb = mean_absolute_error(validation_data['quantity'], predictions_xgb)

# Paso 7:
# Guardamos los modelos con joblib
joblib.dump(pipeline_dummy, 'dummy_regressor_model.pkl')
joblib.dump(pipeline_xgb, 'xgb_regressor_model.pkl')


['xgb_regressor_model.pkl']

In [7]:
# Los printeamos aca para que no colapse
print(f'MAE: DummyRegressor: {mae_dummy}')
print(f'MAE: XGBRegressor: {mae_xgb}')

MAE: DummyRegressor: 13308.134750658153
MAE: XGBRegressor: 2345.606341495692


En el contexto del error, quiere decir que en promedio cada predicción se equivoca en 13308 de cantidades, es decir digo que cierto producto se necesita 30.000, entonces lo más probable es que este equivocado 13.308 unidades de cantidad hacia arriba o abajo.

Luego los resultados de XGB hablan por si solos, al incluir las variables numericas tenemos que existe un error de 2345.60 de cantidades, es decir que si digo que necesito 30.000 de cierto producto, entonces estaria equivocado 2346 hacia arriba o hacia abajo. Lo cual es mucho mejor que que el dummy regressor.



## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [8]:
# Inserte su código acá


xgb_params = {
    'monotone_constraints': (0,0,0,0,0,0,0,0,0,0,-1) # para que tome a al precio con su relacion negativa ()
}


# Partimos de 6 ->
# Lo mismo pero con xgb
pipeline_xgb2 = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**xgb_params))
])


train_x = train_data.drop('quantity', axis=1)
pipeline_xgb2.fit(train_x, train_data['quantity'])
predictions_xgb = pipeline_xgb2.predict(validation_data)

# lo guardamos
mae_xgb2 = mean_absolute_error(validation_data['quantity'], predictions_xgb)

# Paso 7:
# Guardamos los modelos con joblib
joblib.dump(pipeline_xgb2, 'xgb_regressor_model_monotone.pkl')

['xgb_regressor_model_monotone.pkl']

In [9]:
print(mae_xgb2)

2350.76454836918


Al incluir el parametro de relacion monotoma, vemos que empeora los resultados en 2345.60 - 2350.76 = 5, esto nos indica que empeora un poco la prediccion del modelo, por lo que el amigo no tenia razón, el mae aumenta y por tanto el feature extra no ayudo al modelo, tal vez probando con hiperparametros diferentes los resultados serian distintos.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [10]:
# Inserte su código acá
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [11]:
# Inserte su código acá

# Primero hacemos una copia del df

# Empezamos de nuevo desde 6, creando primero la funcion objetivo la cual se va a optimizar
# 6. Lo mismo pero con xgb
def objective(trial):

    # Definimos los parámetros que nos sugieren 
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Definimos los parámetros que nos sugieren para onehot

    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    preprocessor = ColumnTransformer(
    transformers=[
        ('date', date_transformer, ['date']),
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(min_frequency=min_frequency), categorical_features)
    ])

    # Corremos la pipeline de siempre
    pipeline_xgb = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params))
    ])

    pipeline_xgb.fit(train_x, train_data['quantity'])
    predictions_xgb = pipeline_xgb.predict(validation_data)
    # Obtenemos el error que es lo que finalmente queremos optimizar
    mae_xgb = mean_absolute_error(validation_data['quantity'], predictions_xgb)

    return mae_xgb

# Ocupamos el sampler TPESampler 
sampler = optuna.samplers.TPESampler(seed=42)

# Crear un estudio de Optuna con el sampler TPESampler
study = optuna.create_study(sampler=sampler, direction='minimize')

# Ejecutar la optimización durante 5 minutos
study.optimize(objective, timeout=300)

# Obtener los mejores hiperparámetros
best_params = study.best_params

# 7. Guardamos el modelo XGB con los mejores hiperparámetros
best_xgb_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**best_params))
])

# Entrenar el modelo XGBoost con todos los datos
best_xgb_model.fit(train_data.drop('quantity', axis=1) , train_data['quantity'])
best_xgb_model

joblib.dump(best_xgb_model, 'xgb_regressor_model_opti.pkl')

[I 2023-11-18 23:01:57,183] A new study created in memory with name: no-name-7653af1d-5871-4d97-93fd-e03cb5d9c7a3
[I 2023-11-18 23:01:57,986] Trial 0 finished with value: 6830.88193388711 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 6830.88193388711.
[I 2023-11-18 23:01:58,307] Trial 1 finished with value: 2844.377332863088 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 2844.377332863088.
[I 2023-11-18 23:01:58,689] Trial 2 finished with value: 6375.951060398239 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_depth':

['xgb_regressor_model_opti.pkl']

In [12]:
best_params

{'learning_rate': 0.09999351087688865,
 'n_estimators': 905,
 'max_depth': 7,
 'max_leaves': 65,
 'min_child_weight': 5,
 'reg_alpha': 0.4503251925153572,
 'reg_lambda': 0.19946501818744633,
 'min_frequency': 0.1407503651295016}

In [13]:
# Tomamos los paraemtros y lo entrenamos de nuevo para obtener el mae
best_xgb_model_opti = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**best_params))
])

# Entrenar el modelo XGBoost con todos los datos
best_xgb_model_opti.fit(train_data.drop('quantity', axis=1) , train_data['quantity'])

predictions_xgb_opti_pr = best_xgb_model_opti.predict(validation_data)

mae_xgb_opti_pr = mean_absolute_error(validation_data['quantity'], predictions_xgb_opti_pr)
print(mae_xgb_opti_pr)

c:\Users\josei\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:06:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


1973.0746602566442


 El learning rate indica la contribución de cada arbol al modelo general, es una variable que ayuda a evitar el sobre ajuste

 n_estimators indica el numero total de arboles en el ensamblaje

 max_deph indica la profundidad maxima del arbol

 max leaves indica la cantidad maxima de hojas del arbol

 min child weight indica la cantidad de veces que tiene que aparecer una variable como divisora en el arbol

 reg_aplha controla las regulacion L1 ( Lasso ) de parametros entre más bajo, valores de coeficientes más altos se permiten

 reg_lambda controla las regulacion L2 ( Ridge ) de parametros mas bajo, valores de coeficientes mas altos se permiten

 min_frequency indica una frecuencia minima necesaria para que una categoria se considere en el one hot encoding, sirve para no agregar complejidad demas al modelo.
 
 Hace sentido los valores ya que hay que probar todas las posibilidades
 
Los mejores parametros fueron los siguientes:
{'learning_rate': 0.09999351087688865,
 'n_estimators': 905,
 'max_depth': 7,
 'max_leaves': 65,
 'min_child_weight': 5,
 'reg_alpha': 0.4503251925153572,
 'reg_lambda': 0.19946501818744633,
 'min_frequency': 0.1407503651295016}

 Con el siguiente error: 1973.074

 Entrega mejores resultados que el modelo anterior, esto debido a que al entrenar un modelo, puede pasar que los parametros por default no se ajusten bien a los datos, por lo que probar diferentes combinaciones permite que se ajuste mejor y entregue finalmente un error más bajo, ahora bien esto tiene un defecto ya que de alguna forma se esta ajustando según los resultados del test, lo que podria significar una especie de ajuste sesgado en el test. igualmente permite encontrar mejores resultados.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [14]:

# Tuve que implementarlo sin los pipelines por errores con los tipos de la columnas que no me aceptaban, :)
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

def extract_date(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('float')
    df['month'] = df['date'].dt.month.astype('float')
    df['year'] = df['date'].dt.year.astype('float')
    return df.drop('date', axis=1)


# Hacemos el df a trabajar a manos sin el pipeline porque no funcionaba
# Aplicar la transformación de fecha al DataFrame original
df22 = extract_date(df2)

# Obtener columnas numéricas y categóricas
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity']

# Aplicar OneHotEncoder a las columnas categóricas
encoder = OneHotEncoder(dtype='float64')
df_cat_encoded = pd.DataFrame(encoder.fit_transform(df22[categorical_features]).toarray(),
                              columns=encoder.get_feature_names_out(categorical_features))

# Concatenar las columnas numéricas y categóricas codificadas
df22 = pd.concat([df22[['day', 'month', 'year'] + numeric_features], df_cat_encoded], axis=1)

# Empezamos de nuevo desde 6, creando primero la función objetivo que se va a optimizar
# Separamos los datos en los tres conjuntos

#train_data22, test_data22 = train_test_split(df22, test_size=0.3, random_state=42)

# Usar drop para eliminar la columna 'Label' y obtener X
##X_train = train_data22.drop('quantity', axis=1)
#X_test = test_data22.drop('quantity', axis=1)

# La variable Y es la columna 'Label'
#Y_train = train_data22['quantity']
#Y_test = test_data22['quantity']




 
def objective(trial):
    data, target = df22.drop('quantity', axis=1) , df22['quantity']
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        #'n_estimators': trial.suggest_int('n_estimators', 50, 1000), # no se usa en el prunning
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        #'enable_categorical': True, # 
    }

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    model_xgb = xgb.train(xgb_params, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])


    # predice
    preds = model_xgb.predict(dvalid)
    #pred_labels = np.rint(preds)
    #
    mae_xgb = mean_absolute_error(valid_y, preds) 



    # Calcular el error que queremos optimizar (MAE)
    #mae_xgb = mean_absolute_error(validation_data22['quantity'], predictions_xgb)

    #bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    #preds = bst.predict(dvalid)
    #pred_labels = np.rint(preds)



    return mae_xgb

# Ocupamos el sampler TPESampler 
sampler = optuna.samplers.TPESampler(seed=42)

# Crear un estudio de Optuna con el sampler TPESampler
study2 = optuna.create_study(sampler=sampler, direction='minimize')

# Ejecutar la optimización durante 5 minutos
study2.optimize(objective, timeout=300)

# Obtener los mejores hiperparámetros
best_params2 = study2.best_params

# 7. Guardamos el modelo XGB con los mejores hiperparámetros
best_xgb_model2 = XGBRegressor(**best_params2)

# Entrenar el modelo XGBoost con todos los datos
best_xgb_model2.fit(df22.drop('quantity', axis=1) , df22['quantity'])

# Guardar el modelo entrenado
joblib.dump(best_xgb_model2, 'xgb_regressor_model_opti_prunning.pkl')

[I 2023-11-18 23:06:59,992] A new study created in memory with name: no-name-fe60659d-9593-49de-9854-abe187befa11


[0]	validation-rmse:17421.27890
[1]	validation-rmse:16969.79620
[2]	validation-rmse:16540.55548
[3]	validation-rmse:16135.32370
[4]	validation-rmse:15744.63077
[5]	validation-rmse:15385.79936
[6]	validation-rmse:15014.62032
[7]	validation-rmse:14665.60332
[8]	validation-rmse:14320.38468
[9]	validation-rmse:13998.85782


[I 2023-11-18 23:07:00,071] Trial 0 finished with value: 10565.419246035073 and parameters: {'learning_rate': 0.03807947176588889, 'max_depth': 10, 'max_leaves': 73, 'min_child_weight': 3, 'reg_alpha': 0.15601864044243652, 'reg_lambda': 0.15599452033620265}. Best is trial 0 with value: 10565.419246035073.


[0]	validation-rmse:17653.21888
[1]	validation-rmse:17576.19366
[2]	validation-rmse:17499.25168
[3]	validation-rmse:17422.96927
[4]	validation-rmse:17347.17088
[5]	validation-rmse:17272.21130
[6]	validation-rmse:17198.28496
[7]	validation-rmse:17124.55761
[8]	validation-rmse:17052.01143
[9]	validation-rmse:16979.56516


[I 2023-11-18 23:07:00,130] Trial 1 finished with value: 12910.759736286213 and parameters: {'learning_rate': 0.006750277604651747, 'max_depth': 9, 'max_leaves': 60, 'min_child_weight': 4, 'reg_alpha': 0.020584494295802447, 'reg_lambda': 0.9699098521619943}. Best is trial 0 with value: 10565.419246035073.


...

[0]	validation-rmse:16480.03182


[I 2023-11-18 23:11:59,685] Trial 2029 pruned. Trial was pruned at iteration 0.
[I 2023-11-18 23:11:59,874] Trial 2030 pruned. Trial was pruned at iteration 0.


[0]	validation-rmse:17306.33843


[I 2023-11-18 23:12:00,060] Trial 2031 pruned. Trial was pruned at iteration 0.


['xgb_regressor_model_opti_prunning.pkl']

In [19]:
best_params2

{'learning_rate': 0.09991538552886423,
 'max_depth': 9,
 'max_leaves': 95,
 'min_child_weight': 4,
 'reg_alpha': 0.39496783583395445,
 'reg_lambda': 0.03380057854213806}

In [20]:
best_xgb_model2 = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(**best_params2))
])

# Entrenar el modelo XGBoost con todos los datos
best_xgb_model2.fit(train_data.drop('quantity', axis=1) , train_data['quantity'])

predictions_xgb_opti_pr = best_xgb_model2.predict(validation_data)

mae_xgb_opti_pr = mean_absolute_error(validation_data['quantity'], predictions_xgb_opti_pr)
print(mae_xgb_opti_pr)

2344.061584554145


El prunning coniste en podar los arboles con el objeito de simplificar los arboles que se generan tal que permita generalizar más los modelos y al mismo tiempo facilitar al computador el correr de estos algoritmos, entonces el prunning permite detener configuracioens de parametros que no presentan una mejora significativa en encontrar los mejores parametros, entonces deberia finalmente encontrar resultados más generales lo que podría beneficiar al modelo final. 


¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

No lograron mejorar mucho, tal vez hubo mala suerte o el modelo de poda no logra obtener mejores resultados, también al incluir más cosas que tiene que hacer el algortimo no alcanza a probar la misma cantidad de configuracion que sin la poda, lo que hace que 5 minutos no sea suficiente para encontrar una mejor combinación. Finalmente la comparación es:

sin prunnnig 5 minutos = 1973.074

con prunnig 5 minutos = 2344.06




## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [17]:
# Inserte su código acá
#import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

SEED = 42

In [18]:
display(plot_optimization_history(study2))
display(plot_parallel_coordinate(study2))
display(plot_param_importances(study2))

¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

En los trials 2, 13, 21 ,52, 94 ,210, 1132 y 1215 se encuentran modelos mejores, sin embargo en los 3 primeros trials que se mencionan las mejoras son muy altas.

En las coordenadas paralelas podemos ver en que valores se movio cada hiperparametro, por ejemplo el lr tiende a tener valores menores a 0, el max depth prefiere tener valores de  ( en orden) 10, 9 ,8 ,7 y6  . Tambien las max leaves estan entre las 100 y 70 hojas por lo general. El min child weight  tiene una tendencia de 4. Por ultimo el reg alpha prefiere moverse entre 0,7 y 1, y reg lambda entre 0,4 y 0,8. Todas estans tendencias indican para donde se encuentra el mejor hiperparametros.

En cuanto a la importancia de cada hyperparametros tenemos que el lr es el más importante con un 93% de importancia , luego el reg_lambda con un 4%, max leaves con 2%, reg_alpha ocn un 1% y el resto menor a 1%.




## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

## Tabla de modelos

|           | Only dummies | XGB baseline | Modelo XGB Mono | Modelo XGB Optuna | Modelo XGB Optuna + Prunning|
|-----------|-----------|-----------|-----------|-----------|-----------|
| MAE   | 13.308,13   | 2.345,60   | 2.350,76   | 1.973,07   | 2.344,06    |

Observamos que en orden de desempeño tenemos:

1: Modelo XGB Optuna 

2: Modelo XGB Optuna + Prunning

3: XGB baseline

4: Modelo XGB Mono 

5: Only dummies






Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [23]:
# Tomamos los paraemtros y lo entrenamos de nuevo para obtener el mae, lo hice rapido, perdon por no llamar el pickle de manera formal.

predictions_xgb_opti_pr_t = best_xgb_model_opti.predict(test_data)

mae_xgb_opti_pr = mean_absolute_error(test_data['quantity'], predictions_xgb_opti_pr_t)
print(mae_xgb_opti_pr)

1899.9174239761292


Sí, mejoró con respecto a lo que se tenia al probar con el grupo de validación, sin embargo lo esperable es que empeore un poco más los resultados esto debido a que al estar ocupuando el conjunto de validación como referencia pra ir ajustando el modelo, esto deberia hacer que el modelo se sobre ajuste con los datos de validacion y train al mismo tiempo, por eso es importante verificar con otro set de datos el rendimientos para saber si este sobre asjute esta afectando mucho a la predicción, con lo que finalmetne notamos que mejoraron los reusltados por lo que en este caso el modelo logro mantener la generalizacionm con un MAE de 1.899 v/s 1.973

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>